In [1]:
import os
import re
import pandas as pd
from itertools import tee
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pymorphy2
from tqdm import tqdm

DATASET_PATH = "ru_rag_test_dataset_renamed.pkl"  
FILES_FOLDER = "files_cleaned"      
CHUNK_SIZE = 500
TOP_K = 1
MAX_QUERIES = 100 

morph = pymorphy2.MorphAnalyzer()

def remove_accents(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return re.sub(r'\u0301', '', text)

def clean_text(text: str) -> str:
    text = remove_accents(text)
    return text.lower()

def lemmatize(text: str):
    tokens = re.findall(r'\w+', text.lower())
    return [morph.parse(t)[0].normal_form for t in tokens if t.isalpha()]

def ngrams(tokens, n=1):
    if len(tokens) < n:
        return []
    t = tee(tokens, n)
    for i, sub in enumerate(t):
        for _ in range(i):
            next(sub, None)
    return list(zip(*t))

def ngram_overlap(answer, context, n=1):
    a_tokens = lemmatize(answer)
    c_tokens = lemmatize(context)
    a_ngrams = set(ngrams(a_tokens, n))
    c_ngrams = set(ngrams(c_tokens, n))
    if not a_ngrams:
        return 0.0
    return len(a_ngrams & c_ngrams) / len(a_ngrams)

df = pd.read_pickle(DATASET_PATH).head(MAX_QUERIES)
chunk_texts = []
chunk_sources = []

for filename in os.listdir(FILES_FOLDER):
    path = os.path.join(FILES_FOLDER, filename)
    if os.path.isfile(path):
        with open(path, "r", encoding="utf-8") as f:
            text = clean_text(f.read())
            words = text.split()
            for i in range(0, len(words), CHUNK_SIZE):
                chunk = " ".join(words[i:i+CHUNK_SIZE])
                chunk_texts.append(chunk)
                chunk_sources.append(f"{filename}_chunk{i//CHUNK_SIZE}")

vectorizer = TfidfVectorizer()
chunk_vectors = vectorizer.fit_transform(chunk_texts)

def retrieve(query, top_k=TOP_K):
    query_vec = vectorizer.transform([query])
    sims = cosine_similarity(query_vec, chunk_vectors).flatten()
    top_idx = sims.argsort()[-top_k:][::-1]
    return [chunk_texts[i] for i in top_idx], [chunk_sources[i] for i in top_idx]

def rag_answer(query):
    retrieved, _ = retrieve(query, top_k=TOP_K)
    return retrieved[0] if retrieved else ""

uni_overlaps = []
bi_overlaps = []
tri_overlaps = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing queries"):
    query_clean = clean_text(row['question'])
    generated = rag_answer(query_clean)
    reference = clean_text(row['answer'])
    # print(generated)
    # print(reference)

    uni_overlaps.append(ngram_overlap(reference, generated, 1))
    bi_overlaps.append(ngram_overlap(reference, generated, 2))
    tri_overlaps.append(ngram_overlap(reference, generated, 3))

print("Среднее пересечение n-грамм:")
print("Униграммы:", sum(uni_overlaps)/len(uni_overlaps))
print("Биграммы:", sum(bi_overlaps)/len(bi_overlaps))
print("Триграммы:", sum(tri_overlaps)/len(tri_overlaps))

df['uni_overlap'] = uni_overlaps
low_uni = df.sort_values('uni_overlap', ascending=True).head(5)

print("Примеры с минимальным пересечением униграмм:")
for i, row in low_uni.iterrows():
    print(f"Вопрос: {row['question']}")
    print(f"Ответ (референс): {row['answer']}")
    print(f"Сгенерированный контекст: {rag_answer(clean_text(row['question']))}")
    print(f"uni_overlap = {row['uni_overlap']:.3f}")


Processing queries: 100%|██████████| 100/100 [00:35<00:00,  2.83it/s]


Среднее пересечение n-грамм:
Униграммы: 0.6327777777777777
Биграммы: 0.12833333333333333
Триграммы: 0.022857142857142857
Примеры с минимальным пересечением униграмм:
Вопрос: К какой династии относился Ричард Львиное Сердце?
Ответ (референс): Плантагенет
Сгенерированный контекст: убедительно опровергнутую медиевистом джоном джиллингемом, упрекнувшим их в буквальной трактовке традиционных для средневековой историографии реминисценций из ветхого завета. в настоящее время данный вопрос остаётся открытым, поскольку основывается на толковании всего двух неоднозначных упоминаний его отношений с французским королем, и мнения ведущих историков о сексуальной ориентации монарха различаются. во всяком случае, детей, рождённых в браке, у 41-летнего короля не было. жена (с 12 мая 1191, кипр) беренгария наваррская (ок. 1170—1230), дочь санчо vi мудрого, короля наварры брак был бесплоден. внебрачная связь. внебрачный сын — филипп де коньяк (начало 1180-х — после 1201). филипп де коньяк послужил прообр